In [1]:
#%load_ext autoreload
#%autoreload 2
%matplotlib inline
%cd '../HOTS'

/Users/laurentperrinet/quantic/science/HomeHots/HOTS_clone_laurent/HOTS


In [2]:
import os
import pickle
import datetime
from mix_Network import *

dataset = 'nmnist'
records_path = '../Records'
timestr = datetime.datetime.now().date().isoformat()
timestr = '2020-12-21'
verbose = True

%mkdir -p ../Records
%mkdir -p ../Records/EXP_03_NMNIST

homeo = True
sigma = None
pooling = False
homeinv = False
jitter = False
tau = 5
krnlinit = 'first'
nblay = 3
nbclust = 4

ds = 10
nb_train = 7500//ds
nb_test = 2500//ds

In [3]:
def get_nmnist(NbTrainingData, NbTestingData):
    def loadev(nbdigit, loader, learningset):
        timout = []
        xout = []
        yout = []
        polout = []
        labout = []
        for i in range(NbTrainingData):
            events, target = next(iter(loader))
            for iev in range(events.shape[1]):
                    timout.append(events[0][iev][learningset.ordering.find("t")].item())
                    xout.append(events[0][iev][learningset.ordering.find("x")].item())
                    yout.append(events[0][iev][learningset.ordering.find("y")].item())
                    polout.append(events[0][iev][learningset.ordering.find("p")].item())
                    labout.append(target.item())

        eventsout = [xout,yout,timout,polout,labout,learningset.sensor_size,2]
        return eventsout
    
    learningset = tonic.datasets.NMNIST(save_to='../Data/',
                                    train=False,
                                    transform=None)
    loader = tonic.datasets.DataLoader(learningset, shuffle=True)
    
    events_train = loadev(NbTrainingData, loader, learningset)
    events_test = loadev(NbTestingData, loader, learningset)
    
    return events_train, events_test

#### Building matrix for logistic regression
def gather_data(events_in,
                tau_cla=150, # characteristic time of a digit
                sample_events=50, sample_space = 1,
                verbose=False, debug=False):
    tau_cla *= 1e3 # to enter tau in ms
    n_events = len(events_in[0])

    c_int = lambda n, d : ((n - 1) // d) + 1
    
    data = np.zeros((c_int(events_in[-2][0], sample_space),
                     c_int(events_in[-2][1], sample_space),
                     events_in[-1])) #tmp data

    X = np.zeros((c_int(n_events, sample_events), len(data.ravel())))
    y = np.zeros((c_int(n_events, sample_events), ))

    for i_event in range(1, n_events):
        if events_in[2][i_event]<events_in[2][i_event-1]:
            data = np.zeros((c_int(events_in[-2][0], sample_space),
                     c_int(events_in[-2][1], sample_space),
                     events_in[-1])) #tmp data 

        data *= np.exp(-(events_in[2][i_event]-events_in[2][i_event-1])/tau_cla)
        
        x_pos = events_in[0][i_event]//sample_space
        y_pos = events_in[1][i_event]//sample_space
        p = events_in[3][i_event]
        data[int(x_pos), int(y_pos), int(p)] = 1.

        if i_event % sample_events == sample_events//2 :
            if debug:
                print(f'DEBUG {i_event} {i_event//sample_events} ')
                print(f'DEBUG {y[i_event//sample_events]}   ')
                print(f'DEBUG  {events_in[-3][i_event]} ')
            X[i_event//sample_events, :] = data.ravel()
            y[i_event//sample_events] = events_in[-3][i_event]
            
    if verbose: print('Number of events: ' + str(X.shape[0])+' - Number of features: ' + str(X.shape[1]))

    return X, y

def get_events(tau, krnlinit, nblay, nbclust, homeo, records_path, timestr, dataset, sigma, homeinv, jitter, nb_train, nb_test):
    hotshom = network(krnlinit=krnlinit, tau=tau, nblay=nblay, nbclust=nbclust, homeo=homeo, sigma=sigma, homeinv=homeinv, jitter=jitter)
    arch = [hotshom.L[i].kernel.shape[1] for i in range(len(hotshom.L))]
    fname = f'{records_path}/EXP_03_NMNIST/{timestr}_hots_{dataset}_{arch}_{tau}_{homeo}_{sigma}_{homeinv}_{jitter}'
    print(fname)
    if not os.path.isfile(fname+'_model.pkl'):
        loaderhom, order = hotshom.learning1by1(dataset=dataset)
        with open(fname+'_model.pkl', 'wb') as file:
            pickle.dump([hotshom, loaderhom, order], file, pickle.HIGHEST_PROTOCOL)
    else:
        with open(fname+'_model.pkl', 'rb') as file:
            hotshom, loaderhom, order = pickle.load(file)
            
    if not os.path.isfile(fname+f'_evout_{nb_train}_{nb_test}.pkl'):       
        _, loaderhom, out_train = hotshom.running(loaderhom, order, nb_digit = nb_train, LR=True)
        _, loaderhom, out_test = hotshom.running(loaderhom, order, nb_digit = nb_test, LR=True)
        with open(fname+f'_evout_{nb_train}_{nb_test}.pkl', 'wb') as file:
            pickle.dump([out_train, out_test], file, pickle.HIGHEST_PROTOCOL)
    else:
        with open(fname+f'_evout_{nb_train}_{nb_test}.pkl', 'rb') as file:
            out_train, out_test = pickle.load(file)
    return out_train, out_test


# ### Performingxlogistic regression
#
# https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegressionCV.html
#

# from sklearn.linear_model import LogisticRegression as LR
from sklearn.linear_model import LogisticRegressionCV as LR
# 
opts_LR = dict(max_iter=2000, # random_state=0,
               n_jobs=-1, class_weight='balanced')
#opts_LR['Cs'] = 5
opts_LR['Cs'] = 32
# TODO for a publication use 100 from 10^{-10} to 10

from sklearn.model_selection import train_test_split

def tic():
    global ttic
    ttic = time.time()
def toc():
    print(f'Done in {time.time() - ttic:.3f} s')


# Performing logistic regression

https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegressionCV.html


## Performing logistic regression on raw input spikes

In [4]:
tic()
events_train, events_test = get_nmnist(nb_train, nb_test)
toc()

Using downloaded and verified file: ../Data/nmnist_test.zip
Extracting ../Data/nmnist_test.zip to ../Data/
Done in 215.650 s


In [5]:
tic()
X_train, y_train = gather_data(events_train, verbose=verbose)
toc()

Number of events: 62967 - Number of features: 2312
Done in 22.857 s


In [6]:
tic()
lr = LR(**opts_LR).fit(X_train, y_train)
print(f'Classification score for raw input is {lr.score(X_train, y_train):.3f} (train)')
toc()

Classification score for raw input is 0.926 (train)
Done in 4785.955 s


In [7]:
tic()
X_test, y_test = gather_data(events_test, verbose=verbose)
print(f'Classification score for raw input is {lr.score(X_test, y_test):.3f} (test)')
toc()

Number of events: 62969 - Number of features: 2312
Classification score for raw input is 0.795 (test)
Done in 20.930 s


## Performing logistic regression with or without homeostasis

In [8]:
for homeo in [False, True]:
    print(40*'-')
    print(f'homeo={homeo}')
    print(40*'-')
    events_train_o, events_test_o = get_events(tau, krnlinit, nblay, nbclust, homeo, records_path, timestr, dataset, sigma, homeinv, jitter, nb_train, nb_test)
        
    X_train, y_train = gather_data(events_train_o, verbose=verbose)

    lr = LR(**opts_LR, verbose=verbose).fit(X_train, y_train)
    print(f'Classification score for homeo = {homeo} is {lr.score(X_train, y_train):.3f} (train)')
    
    X_test, y_test = gather_data(events_test_o, verbose=verbose)
    print(f'Classification score for homeo = {homeo} is {lr.score(X_test, y_test):.3f} (test)')

----------------------------------------
homeo=False
----------------------------------------
../Records/EXP_03_NMNIST/2020-12-21_hots_nmnist_[4, 8, 16]_5_False_None_False_False
Using downloaded and verified file: ../Data/nmnist_test.zip
Extracting ../Data/nmnist_test.zip to ../Data/


  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

FileNotFoundError: [Errno 2] No such file or directory: '../Records/EXP_03_NMNIST/2020-12-21_hots_nmnist_[4, 8, 16]_5_False_None_False_False_model.pkl'

### Performing logistic regression with different solvers

In [9]:
for homeo in [False, True]:
    print(40*'-')
    print(f'{homeo=}')
    print(40*'-')
    for solver, penalty in zip(['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'], ['l2', 'l2', 'l1', 'l2', 'l1']):
        events_train_o, events_test_o = get_events(tau, krnlinit, nblay, nbclust, homeo, records_path, timestr, dataset, sigma, homeinv, jitter, nb_train, nb_test)
        tic = time.time()
        X_train, y_train = gather_data(events_train_o, verbose=verbose)
        lr = LR(solver=solver, penalty=penalty, verbose=verbose, **opts_LR).fit(X_train, y_train)
        print(f'Classification score for {homeo=} is {lr.score(X_train, y_train):.3f} (train)')
        print(f'Fit done in {time.time() -tic:.3f} seconds for {solver=}')
        X_test, y_test = gather_data(events_test_o, verbose=verbose)
        print(f'Classification score for {homeo=} / {solver=} is {lr.score(X_test, y_test):.3f} (test)')

----------------------------------------
homeo=False
----------------------------------------
../Records/EXP_03_NMNIST/2020-12-21_hots_nmnist_[4, 8, 16]_5_False_None_False_False
Using downloaded and verified file: ../Data/nmnist_test.zip
Extracting ../Data/nmnist_test.zip to ../Data/


  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

FileNotFoundError: [Errno 2] No such file or directory: '../Records/EXP_03_NMNIST/2020-12-21_hots_nmnist_[4, 8, 16]_5_False_None_False_False_model.pkl'

In [10]:
for homeo in [False, True]:
    print(40*'-')
    print(f'{homeo=}')
    print(40*'-')
    events_train_o, events_test_o = get_events(tau, krnlinit, nblay, nbclust, homeo, records_path, timestr, dataset, sigma, homeinv, jitter, nb_train, nb_test)

    for tau_cla in 150 * np.logspace(-1, 1, 7, base=4):

        X_train, y_train = gather_data(events_train_o, tau_cla=tau_cla, verbose=verbose)
        lr = LR(**opts_LR, verbose=verbose).fit(X_train, y_train)
        print(f'Classification score for {tau_cla=:.3f} is {lr.score(X_train, y_train):.3f} (train)')
        X_test, y_test = gather_data(events_test_o, tau_cla=tau_cla, verbose=verbose)
        print(f'Classification score for {tau_cla=:.3f} is {lr.score(X_test, y_test):.3f} (test)')

----------------------------------------
homeo=False
----------------------------------------
../Records/EXP_03_NMNIST/2020-12-21_hots_nmnist_[4, 8, 16]_5_False_None_False_False
Using downloaded and verified file: ../Data/nmnist_test.zip
Extracting ../Data/nmnist_test.zip to ../Data/


  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

FileNotFoundError: [Errno 2] No such file or directory: '../Records/EXP_03_NMNIST/2020-12-21_hots_nmnist_[4, 8, 16]_5_False_None_False_False_model.pkl'

In [11]:
for homeo in [False, True]:
    print(40*'-')
    print(f'{homeo=}')
    print(40*'-')
    events_train_o, events_test_o = get_events(tau, krnlinit, nblay, nbclust, homeo, records_path, timestr, dataset, sigma, homeinv, jitter, nb_train, nb_test)

    for sample_space in [4, 2, 1]:
        X_train, y_train = gather_data(events_train_o, sample_space=sample_space, verbose=verbose)
        lr = LR(**opts_LR, verbose=verbose).fit(X_train, y_train)
        print(f'Classification score for {sample_space=} is {lr.score(X_train, y_train):.3f} (train)')
        X_test, y_test = gather_data(events_test_o, sample_space=sample_space, verbose=verbose)
        print(f'Classification score for {sample_space=} is {lr.score(X_test, y_test):.3f} (test)')


----------------------------------------
homeo=False
----------------------------------------
../Records/EXP_03_NMNIST/2020-12-21_hots_nmnist_[4, 8, 16]_5_False_None_False_False
Using downloaded and verified file: ../Data/nmnist_test.zip
Extracting ../Data/nmnist_test.zip to ../Data/


  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

FileNotFoundError: [Errno 2] No such file or directory: '../Records/EXP_03_NMNIST/2020-12-21_hots_nmnist_[4, 8, 16]_5_False_None_False_False_model.pkl'

In [12]:
tauz = np.array([1e-5, 1e-4, 2e-4 ,3e-4, 4e-4, 5e-4, 6e-4, 7e-4,
                 8e-4, 9e-4, 1e-3, 2e-3, 2.5e-3, 3e-3, 4e-3, 5e-3, 
                 1e-2, 2e-2])
tauz *=1e3
for homeo in [False, True]:
    print(40*'-')
    print(f'{homeo=}')
    print(40*'-')
    for tau_ in tauz:
        events_train_o, events_test_o = get_events(tau, krnlinit, nblay, nbclust, homeo, records_path, timestr, dataset, sigma, homeinv, jitter, nb_train, nb_test)
        
        X_train, y_train = gather_data(events_train_o, verbose=verbose)
        lr = LR(**opts_LR, verbose=verbose).fit(X_train, y_train)
        print(f'Classification score for {homeo=} & {tau_=:.6f} is {lr.score(X_train, y_train):.3f} (train)')
        X_test, y_test = gather_data(events_test_o, verbose=verbose)
        print(f'Classification score for {homeo=} & {tau_=:.6f} is {lr.score(X_test, y_test):.3f} (test)')


----------------------------------------
homeo=False
----------------------------------------
../Records/EXP_03_NMNIST/2020-12-21_hots_nmnist_[4, 8, 16]_5_False_None_False_False
Using downloaded and verified file: ../Data/nmnist_test.zip
Extracting ../Data/nmnist_test.zip to ../Data/


  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

FileNotFoundError: [Errno 2] No such file or directory: '../Records/EXP_03_NMNIST/2020-12-21_hots_nmnist_[4, 8, 16]_5_False_None_False_False_model.pkl'